In [1]:
!pip -q install git+https://github.com/IRTIZA-ZAIDI/irti_custom_models.git

  Preparing metadata (setup.py) ... done


In [2]:
from irti_custom import AutoModelForMaskedSeqModeling
print("import ok", AutoModelForMaskedSeqModeling)


import ok <class 'irti_custom.modeling_custom.AutoModelForMaskedSeqModeling'>


In [3]:
import os
import shutil
import torch

from irti_custom import AutoModelForMaskedSeqModeling

torch.manual_seed(0)

base = "bert-base-uncased"
num_labels = 15
task = "multilabel"

# Provide explicit label maps to ensure they persist across save/load
id2label = {i: f"LBL_{i}" for i in range(num_labels)}
label2id = {v: k for k, v in id2label.items()}

model = AutoModelForMaskedSeqModeling.from_pretrained(
    base,
    num_labels=num_labels,
    task=task,
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=False,
    strict_mlm_check=False,
)
model.eval()

B, T, M = 2, 8, 2
input_ids = torch.randint(0, 100, (B, T))
attention_mask = torch.ones_like(input_ids)
mask_indices = torch.tensor([[1, 5], [2, 6]], dtype=torch.long)

out = model(input_ids=input_ids, attention_mask=attention_mask, mask_indices=mask_indices)
assert out.logits.shape == (B, M, num_labels), out.logits.shape

# loss path: multilabel expects [B, M, L]
labels = torch.zeros((B, M, num_labels))
labels[0, 1, 0] = -100  # ignore second detection for sample 0
out2 = model(
    input_ids=input_ids,
    attention_mask=attention_mask,
    mask_indices=mask_indices,
    labels=labels,
)
assert out2.loss is not None, "loss should be computed for multilabel"
print("BERT multilabel forward+loss OK")

# save & reload
save_dir = "./ckpt_test_bert_multilabel"
if os.path.exists(save_dir):
    shutil.rmtree(save_dir)
model.save_pretrained(save_dir)

model_re = AutoModelForMaskedSeqModeling.from_pretrained(save_dir)
model_re.eval()

# verify num_labels + label maps survived reload
assert model_re.config.num_labels == num_labels, (model_re.config.num_labels, num_labels)
assert int(max(model_re.config.id2label.keys())) == num_labels - 1
assert model_re.config.id2label[0].startswith("LBL_")

out_re = model_re(input_ids=input_ids, attention_mask=attention_mask, mask_indices=mask_indices)
assert out_re.logits.shape == (B, M, num_labels), out_re.logits.shape
print("BERT multilabel save/reload OK")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


BERT multilabel forward+loss OK


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/205 [00:00<?, ?it/s]

BERT multilabel save/reload OK


In [4]:
import os
import shutil
import torch
from irti_custom import AutoModelForMaskedSeqModeling

torch.manual_seed(0)

base = "bert-base-uncased"
num_labels = 5
task = "multiclass"

model = AutoModelForMaskedSeqModeling.from_pretrained(
    base,
    num_labels=num_labels,
    task=task,
    trust_remote_code=False,
    strict_mlm_check=False,
)
model.eval()

B, T, M = 2, 10, 3
input_ids = torch.randint(0, 100, (B, T))
attention_mask = torch.ones_like(input_ids)
mask_indices = torch.tensor([[1, 7, 8], [2, 6, 9]], dtype=torch.long)

# multiclass labels: [B, M], -100 ignore
labels = torch.tensor([[1, 2, -100], [3, 4, 0]], dtype=torch.long)

out = model(
    input_ids=input_ids,
    attention_mask=attention_mask,
    mask_indices=mask_indices,
    labels=labels,
)
assert out.logits.shape == (B, M, num_labels), out.logits.shape
assert out.loss is not None, "loss should be computed for multiclass"
print("BERT multiclass forward+loss OK")

# save & reload
save_dir = "./ckpt_test_bert_multiclass"
if os.path.exists(save_dir):
    shutil.rmtree(save_dir)
model.save_pretrained(save_dir)

model_re = AutoModelForMaskedSeqModeling.from_pretrained(save_dir)
model_re.eval()

out_re = model_re(input_ids=input_ids, attention_mask=attention_mask, mask_indices=mask_indices)
assert out_re.logits.shape == (B, M, num_labels), out_re.logits.shape
print("BERT multiclass save/reload OK")


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


BERT multiclass forward+loss OK


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/205 [00:00<?, ?it/s]

BERT multiclass save/reload OK


In [5]:
import torch
from irti_custom import AutoModelForMaskedSeqModeling

base = "jhu-clsp/mmBERT-base"
num_labels = 15
task = "multilabel"

model = AutoModelForMaskedSeqModeling.from_pretrained(
    base,
    num_labels=num_labels,
    task=task,
    trust_remote_code=True,
    strict_mlm_check=True,
)
model.eval()

B, T, M = 2, 8, 2
input_ids = torch.randint(0, 100, (B, T))
attention_mask = torch.ones_like(input_ids)
mask_indices = torch.tensor([[1, 5], [2, 6]], dtype=torch.long)

out = model(input_ids=input_ids, attention_mask=attention_mask, mask_indices=mask_indices)
assert out.logits.shape == (B, M, num_labels), out.logits.shape
print("mmBERT forward OK")

# save & reload
save_dir = "./ckpt_test_mmbt"
model.save_pretrained(save_dir)
model2 = AutoModelForMaskedSeqModeling.from_pretrained(save_dir)
out2 = model2(input_ids=input_ids, attention_mask=attention_mask, mask_indices=mask_indices)
assert out2.logits.shape == (B, M, num_labels), out2.logits.shape
print("mmBERT save/reload OK")


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/138 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.embeddings.tok_embeddings.weight to decoder.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


model.safetensors:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

...

ModernBertModel LOAD REPORT from: jhu-clsp/mmBERT-base
Key               | Status     |  | 
------------------+------------+--+-
decoder.bias      | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 
decoder.weight    | UNEXPECTED |  | 
head.dense.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


mmBERT forward OK


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/140 [00:00<?, ?it/s]

mmBERT save/reload OK


In [6]:
model

AutoModelForMaskedSeqModeling(
  (modernbert): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(256000, 768, padding_idx=0)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, bias=False)
        )
      )
      (1-21): 21 x ModernBertEncoderLayer(
        (at